Подключим необходимые библиотеки

In [1]:
library(Biobase)
library(GEOquery)
library(biomaRt)

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite B

Считаем данные эскперимента

In [2]:
gset <- getGEO("GSE26728", GSEMatrix =TRUE, getGPL=FALSE)
if (length(gset) > 1) idx <- grep("GPL7042", attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

Found 1 file(s)
GSE26728_series_matrix.txt.gz
Parsed with column specification:
cols(
  ID_REF = col_character(),
  GSM658075 = col_double(),
  GSM658076 = col_double(),
  GSM658077 = col_double(),
  GSM658078 = col_double(),
  GSM658079 = col_double(),
  GSM658080 = col_double(),
  GSM658081 = col_double(),
  GSM658082 = col_double(),
  GSM658083 = col_double(),
  GSM658084 = col_double(),
  GSM658085 = col_double(),
  GSM658086 = col_double(),
  GSM658087 = col_double(),
  GSM658088 = col_double(),
  GSM658089 = col_double(),
  GSM658090 = col_double(),
  GSM658091 = col_double(),
  GSM658092 = col_double()
)


Посмотрим на матрицу экспрессии генов. Здесь названия строк - идентификаторы пробы, названия столбцов - особи.

In [3]:
ex_mat <- exprs(gset)

In [4]:
head(ex_mat)

,GSM658075,GSM658076,GSM658077,GSM658078,GSM658079,GSM658080,GSM658081,GSM658082,GSM658083,GSM658084,GSM658085,GSM658086,GSM658087,GSM658088,GSM658089,GSM658090,GSM658091,GSM658092
A_51_P100034,12.630,11.899,12.393,12.539,12.368,12.361,12.332,12.148,12.032,11.955,12.222,12.202,12.107,12.529,11.838,12.054,12.223,12.343
A_51_P100063,5.828,6.598,5.984,5.986,6.089,6.163,6.384,6.410,5.984,6.158,6.031,5.833,6.441,7.814,6.107,7.017,5.853,6.656
A_51_P100099,7.638,7.635,7.216,7.767,7.516,7.378,7.171,7.558,7.571,7.607,7.523,7.628,7.461,7.321,7.792,7.577,7.412,7.357
A_51_P100155,10.947,10.582,10.406,11.054,10.770,10.402,10.610,10.512,10.492,10.485,10.839,10.535,10.674,10.333,10.622,10.302,10.339,10.572
A_51_P100174,6.632,6.089,6.012,6.190,6.260,6.131,5.993,6.274,6.002,6.011,6.124,6.062,6.375,6.087,6.152,6.202,6.209,6.172
A_51_P100181,6.655,6.788,6.584,6.822,6.742,6.782,6.805,6.822,6.734,6.954,6.777,6.859,6.684,6.600,6.779,7.029,6.616,6.715


In [5]:
ag_probs <- rownames(ex_mat)
samples <- colnames(ex_mat)

Проаннотируем пробы, чтобы получить информацию об исследуемых генах. Для этого воспользуемся пакетом BiomaRT.

In [8]:
mart <- useDataset("mmusculus_gene_ensembl", useMart("ensembl"))
gene_list <- getBM(attributes= c("agilent_wholegenome", "ensembl_gene_id"), values=ag_probs, filter = "agilent_wholegenome", mart= mart)

Cache found


In [9]:
head(gene_list)

agilent_wholegenome,ensembl_gene_id
A_51_P101086,ENSMUSG00000024276
A_51_P122981,ENSMUSG00000104526
A_51_P119633,ENSMUSG00000067199
A_51_P129480,ENSMUSG00000095675
A_51_P133792,ENSMUSG00000031145
A_51_P105755,ENSMUSG00000012640


Посмотрим все ли пробы получилось проаннотировать. Сравним длины исходных и полученных данных:

In [10]:
number_of_not_converted <- length(ag_probs) - length(gene_list$agilent_wholegenome)
number_of_not_converted

[1] 1761

Для 1761 элемента не нашлось соответствий в базе данных. 
Дальнейшие действия будем выполнять без их учета.

Известно, что первые 6 особей соответствуют группе "control", следующие - "BPA at 50" и последние 6 - "BPA at 5000". Найдем средние значения экспрессии генов для каждой из групп.

In [11]:
means_for_genes = data.frame(gene=unique(gene_list$ensembl_gene_id))
means_for_genes$control <- rep(0, nrow(means_for_genes))
means_for_genes$bpa_at_50 <- rep(0, nrow(means_for_genes))
means_for_genes$bpa_at_5000 <- rep(0, nrow(means_for_genes))

In [12]:
for (i in 1:nrow(gene_list)){
    probs_for_gene <- as.vector(gene_list[gene_list$ensembl_gene_id == gene_list$ensembl_gene_id[i],]$agilent_wholegenome)
    means_for_genes[means_for_genes$gene == gene_list$ensembl_gene_id[i],]$control = mean(ex_mat[probs_for_gene, c(1:6)]) / length(probs_for_gene)
    means_for_genes[means_for_genes$gene == gene_list$ensembl_gene_id[i],]$bpa_at_50 = mean(ex_mat[probs_for_gene, c(7:12)]) / length(probs_for_gene)
    means_for_genes[means_for_genes$gene == gene_list$ensembl_gene_id[i],]$bpa_at_5000 = mean(ex_mat[probs_for_gene, c(13:18)]) / length(probs_for_gene)
}

In [13]:
means_for_genes

gene,control,bpa_at_50,bpa_at_5000
ENSMUSG00000024276,3.503208,3.543042,3.496792
ENSMUSG00000104526,6.938000,6.886667,7.115833
ENSMUSG00000067199,4.430417,4.150042,4.361042
ENSMUSG00000095675,6.663333,6.606833,6.965167
ENSMUSG00000031145,7.433667,7.355167,7.270333
ENSMUSG00000012640,2.208241,2.200611,2.205481
ENSMUSG00000028062,13.264333,13.302667,13.122167
ENSMUSG00000068566,3.322625,3.390000,3.285542
ENSMUSG00000039826,4.968250,5.032208,5.050667
ENSMUSG00000051147,10.281833,10.246167,10.291500


Сохраним результат в файл

In [14]:
write.table(means_for_genes, "../data/means_for_genes.txt", row.names = FALSE, col.names = TRUE, sep='\t', quote=FALSE)

Так же запишем в файл матрицу экспрессии для использования в дальнейшем для построения дерева решений.

In [15]:
df = as.data.frame(t(ex_mat))
df$Group <- c(rep(0, 6), rep(1, 6), rep(2, 6))

In [16]:
write.table(df, "../data/expr_matrix.txt", row.names = TRUE, col.names = TRUE, sep='\t', quote=FALSE)